In [1]:
import pandas as pd
data = pd.read_csv('traindata2')

In [2]:
alltext = []
for i in data['text']:
    tl = i.split(' ')
    alltext.append(tl)

In [3]:
from gensim.corpora import Dictionary
dct = Dictionary(alltext)

In [4]:
from keras.preprocessing import sequence
X = [dct.doc2idx(txt) for txt in alltext]
X =sequence.pad_sequences(X, padding='post', value=len(dct))

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,data.iloc[:,-1],test_size=0.3,random_state=0)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Convolution1D
from keras.layers import MaxPool1D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization

In [9]:
model = Sequential()
model.add(Embedding(len(dct)+1, 300, input_length=X.shape[1]))
model.add(Convolution1D(256, 3, padding='same'))
model.add(MaxPool1D(3,3,padding='same'))
model.add(Convolution1D(128, 3, padding='same'))
model.add(MaxPool1D(3,3,padding='same'))
model.add(Convolution1D(64, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(BatchNormalization()) # (批)規範化層
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(4,activation='softmax'))
model.summary()
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1337, 300)         8018400   
                                                                 
 conv1d_3 (Conv1D)           (None, 1337, 256)         230656    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 446, 256)         0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 446, 128)          98432     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 149, 128)         0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 149, 64)          

In [10]:
model.fit(x=X_train,y=y_train, validation_split=0.1, epochs=10, verbose=1)

Epoch 1/10
9/9 [==============================] - 9s 791ms/step - loss: 3.9688 - accuracy: 0.2414 - val_loss: 1.9325 - val_accuracy: 0.0667
Epoch 2/10
9/9 [==============================] - 6s 716ms/step - loss: 2.4531 - accuracy: 0.4521 - val_loss: 1.8891 - val_accuracy: 0.2333
Epoch 3/10
9/9 [==============================] - 6s 713ms/step - loss: 1.4838 - accuracy: 0.5824 - val_loss: 1.7458 - val_accuracy: 0.1667
Epoch 4/10
9/9 [==============================] - 6s 714ms/step - loss: 0.9710 - accuracy: 0.7548 - val_loss: 2.0948 - val_accuracy: 0.4333
Epoch 5/10
9/9 [==============================] - 6s 691ms/step - loss: 0.3816 - accuracy: 0.8697 - val_loss: 2.1934 - val_accuracy: 0.2000
Epoch 6/10
9/9 [==============================] - 6s 698ms/step - loss: 0.5018 - accuracy: 0.8506 - val_loss: 2.2275 - val_accuracy: 0.2333
Epoch 7/10
9/9 [==============================] - 6s 713ms/step - loss: 0.2826 - accuracy: 0.9157 - val_loss: 2.2551 - val_accuracy: 0.2000
Epoch 8/10
9/9 [====

In [11]:
model.evaluate(X_test, y_test ,batch_size=10)

13/13 [==============================] - 1s 70ms/step - loss: 1.7476 - accuracy: 0.1840


[1.7475799322128296, 0.18400000035762787]

In [12]:
model.predict(X_test)

array([[0.39709786, 0.1048238 , 0.2780102 , 0.2200682 ],
       [0.5136466 , 0.22445402, 0.21271071, 0.0491887 ],
       [0.2721783 , 0.056137  , 0.4074906 , 0.2641942 ],
       [0.4068669 , 0.11630963, 0.27662548, 0.20019801],
       [0.3156628 , 0.03735161, 0.3250502 , 0.32193542],
       [0.4200591 , 0.11149372, 0.4368507 , 0.03159647],
       [0.3605171 , 0.04761373, 0.41345882, 0.1784103 ],
       [0.4398962 , 0.07843341, 0.4590999 , 0.0225705 ],
       [0.4276546 , 0.15303561, 0.3926449 , 0.0266649 ],
       [0.41218215, 0.0895384 , 0.4716112 , 0.02666824],
       [0.28268397, 0.04655758, 0.38378555, 0.28697288],
       [0.43221906, 0.36042836, 0.13519874, 0.07215378],
       [0.34901562, 0.084714  , 0.321373  , 0.2448974 ],
       [0.62054217, 0.14501259, 0.16065814, 0.07378707],
       [0.2702517 , 0.06346449, 0.42651862, 0.23976512],
       [0.4469162 , 0.07508601, 0.44453743, 0.03346046],
       [0.29403564, 0.14333615, 0.31729105, 0.24533725],
       [0.33091527, 0.06287672,

In [ ]:
from keras.models import load_model
model.save('kerassave')
newmodel = load_model('kerassave')